# Extract monthly outputs from the GLORYS "Global Ocean Physics Reanalysis"

This script downloads selected variables from the GLORYS "Global Ocean Physics Reanalysis" data product. The default variables to downloaded include sea surface elevation, and water temperature, salinity, u-velocity, and v-velocity at all model depths for any selected ocean sub-area and for any selected time period between 1/1993 to 12/2020. Other variables for ice are also available to add to the data extraction. 

GLORYS data are downloaded from the following link and saved in netcdf files on your google drive:

https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_001_030/download

___

by Greg Pelletier | gjpelletier@gmail.com | https://github.com/gjpelletier/get_glorys

___

INSTRUCTIONS

Before using this script, it is first necessary to establish a free account with https://data.marine.copernicus.eu/ 
Your username will be assigned when you establish your account. Your password should not include special characters.

Specify the following in the code sections below:
  - list of variables to be extracted in any combination, e.g. var_list = ["so","thetao","uo","vo","zos"]
  - west, east, south, and north extent of the ocean sub-area where data will be extracted
  - name of the OUTPUT_DIRECTORY where the hycom data will be saved as output
  - the date_start and number_of_months of the time period to be extracted (between 1/1993 and 12/2020)
  - the min and max depths (dep_min and dep_max) (between 0 and 5728m)

During execution you should see the progress of each monthly file that is extracted during the period of interest from beginning to end. Each nc file name has the format glorys_yyyy_MM.nc to indicate the date stamp


___

Install the motuclient package that is needed for this script:


In [1]:
pip install motuclient==1.8.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import the required packages for this script:

In [2]:
import os
import sys
from datetime import *
import time
from socket import timeout
from dateutil.relativedelta import relativedelta
import calendar     # calendar.monthrange(year, month) returns weekday (0-6 ~ Mon-Sun) and number of days (28-31) for year, month.

# aditional packages needed per https://help.marine.copernicus.eu/en/articles/5211063-how-to-use-the-motuclient-within-python-environment:
import getpass
import motuclient

Mount your google drive folder to make it possible to store the output nc files in your google drive:

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Specify the name of the OUTPUT_DIRECTORY folder where the output files will be saved in your mounted google drive folder. Edit the name of OUTPUT_DIRECTORY below to use any name you want, as long as you start the name with 'drive/MyDrive/' if you are using Google Colab. This subfolder will be created by the script later if it does not already exist in your google drive:

In [3]:
OUTPUT_DIRECTORY = 'drive/MyDrive/Colab Notebooks/glorys/'   # include the ending '/' 

Specify the date_start in ISO format for the starting datetime when the data to be extracted (between 1/1993 and 12/2020). 

The datetime format must be 'YYYY-MM-DD hh:mm:ss'. The starting day should be 01, and the starting hh:mm:ss should be 00:00:00

Also specify the number_of_months to be downloaded. There will be separate output nc files downloaded for each date in the number_of_months. For example, if number_of_months=12 there will be 12 nc files, one file for each month.

Each output nc file name will be generated by the script and will have the format glorys_yyyy_MM.nc to indicate the date stamp. 

Note that the download time for each nc file will depend on the size of the area being extracted. The default area below typically takes less than 1 minute to download each monthly nc file.



In [5]:
# select dates between 1/1993 and 12/2020
date_start = '2020-01-01 00:00:00'      
number_of_months  = 12                     

Specify spatial limits (default below is Parker MacCready's boundary of the LiveOcean model):

In [6]:
north = 53              # -90 to 90 degN          
south = 39              # -90 to 90 degN
west = -131             # -180 to 180 degE
east = -122             # -180 to 180 degE

Edit the following var_list as needed to download any subset of these available variables. Any of the following variables may be included in the var_list:

- so = Seawater salinity [10^-3]
- thetao = Seawater potential temperature [°C]
- uo = Eastward seawater velocity [m/s] 
- vo = Northward seawater velocity [m/s]
- zos = Sea surface height above geoid [m]
- bottomT = sea water potential temperature at sea floor [°C]
- mlotst = Ocean mixed layer thickness defined by sigma theta [m]
- siconc = Sea ice area fraction
- sithick = Sea ice thickness [m]
- usi = Eastward sea ice velocity [m/s]
- vsi = Northward sea ice velocity [m/s]


In [7]:
var_list = ["so","thetao","uo","vo","zos"] 

Specify the min and max depths to download (between 0 - 5728m):

In [8]:
# Specify the min and max depths to download (0 - 5728m)
dep_min = 0
dep_max = 5728

Make a function to create a directory if it does not already exist:

In [9]:
def ensure_dir(file_path):
    # create a folder if it does not already exist
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

Define a class MotuOptions that will be used to parse the motuclient:

In [10]:
class MotuOptions:
    def __init__(self, attrs: dict):
        super(MotuOptions, self).__setattr__("attrs", attrs)

    def __setattr__(self, k, v):
        self.attrs[k] = v

    def __getattr__(self, k):
        try:
            return self.attrs[k]
        except KeyError:
            return None

Make a function to extract the glorys data during the loop through a list of all datetimes to be extracted:

In [11]:
def get_extraction(data_request_options_dict):
    # get the data and save as a netcdf file
    counter = 1
    got_file = False
    while (counter <= 10) and (got_file == False):
        print('  Attempting to get data, counter = ' + str(counter))
        tt0 = time.time()
        try:
            motuclient.motu_api.execute_request(MotuOptions(data_request_options_dict))
        except timeout:
            print('  *Socket timed out, trying again')
        except:
            print('  *Something went wrong, trying again')      
        else:
            got_file = True
            print('  Downloaded data')
        print('  Time elapsed: %0.1f seconds' % (time.time() - tt0))
        counter += 1
    if got_file:
        result = 'success'
    else:
        result = 'fail'
    return result

Make a dt_list of all of the dates to extract from glorys:

In [12]:
base = datetime.fromisoformat(date_start)
ndt = number_of_months
dt_list = []
dt_list = [base + relativedelta(months = x) for x in range(ndt)]

Before using this script, it is first necessary to establish a free account with https://data.marine.copernicus.eu/ Your username will be assigned when you establish your account. Your password should not include special characters. 

Enter your username and password at the prompts (press enter after inputting your username to get the password prompt, then press enter again after inputting the password). 

In [13]:
USERNAME = input('Enter your username: ')
PASSWORD = getpass.getpass('Enter your password: ')

Enter your username: gpelletier
Enter your password: ··········


Make a template dictionary that will used by motuclient and will be updated with new dates and output filenames during each iteration of the loop through days:

In [14]:
data_request_options_dict_manual = {
    "service_id": "GLOBAL_MULTIYEAR_PHY_001_030-TDS",
    "product_id": "cmems_mod_glo_phy_my_0.083_P1M-m",
    "date_min": " ",
    "date_max": " ",
    "longitude_min": float(west),
    "longitude_max": float(east),
    "latitude_min": float(south),
    "latitude_max": float(north),
    "depth_min": float(dep_min),
    "depth_max": float(dep_max),
    "variable": var_list,
    "motu": "https://my.cmems-du.eu/motu-web/Motu",
    "out_dir": OUTPUT_DIRECTORY,
    "out_name": " ",
    "auth_mode": "cas",
    "user": USERNAME,
    "pwd": PASSWORD
}

Loop through all of the dates in dt_list and download all of the nc files for the number_of_months of data:

In [15]:
out_dir = OUTPUT_DIRECTORY                   # specify output directory adding the ending '/'
ensure_dir(out_dir)                         # make sure the output directory exists, make one if not
f = open(out_dir + 'log.txt', 'w+')         # open log of successful downloads
print('\n** Working on GLORYS extraction **')
f.write('\n\n** Working on GLORYS extraction **')
tt1 = time.time()                           # tic for total elapsed time
force_overwrite = True                      # overwrite any already existing nc files in the output folder that have the same names
for dt in dt_list:
    out_fn = datetime.strftime(dt, 'glorys_%Y_%m') + '.nc'
    dstr_min = dt.strftime('%Y-%m-01 00:00:00')
    dd_str = str(calendar.monthrange(int(dt.strftime('%Y')), int(dt.strftime('%m')))[1])  # string of last day in this month
    dstr_max = dt.strftime('%Y-%m-'+dd_str+' 23:59:59')
    data_request_options_dict_manual["out_name"] = out_fn
    data_request_options_dict_manual["date_min"] = dstr_min
    data_request_options_dict_manual["date_max"] = dstr_max
    print(out_dir + out_fn)
    if os.path.isfile(out_fn):
        if force_overwrite:
            os.remove(out_fn)
    if not os.path.isfile(out_fn):
        result = get_extraction(data_request_options_dict_manual)
        f.write('\n ' + datetime.strftime(dt, '%Y_%m') + ' ' + result)
           
# - - -
# final message
totmin = (time.time() - tt1)/60             # total time elapsed for loop over all datetimes in minutes
print('')
print('All downloads are completed.')
print('Total time elapsed: %0.1f minutes' % totmin)
f.close()       # close log of successful downloads


** Working on GLORYS extraction **
drive/MyDrive/Colab Notebooks/glorys/glorys_2020_01.nc
  Attempting to get data, counter = 1
  Downloaded data
  Time elapsed: 41.8 seconds
drive/MyDrive/Colab Notebooks/glorys/glorys_2020_02.nc
  Attempting to get data, counter = 1
  Downloaded data
  Time elapsed: 41.5 seconds
drive/MyDrive/Colab Notebooks/glorys/glorys_2020_03.nc
  Attempting to get data, counter = 1
  Downloaded data
  Time elapsed: 40.3 seconds
drive/MyDrive/Colab Notebooks/glorys/glorys_2020_04.nc
  Attempting to get data, counter = 1
  Downloaded data
  Time elapsed: 41.5 seconds
drive/MyDrive/Colab Notebooks/glorys/glorys_2020_05.nc
  Attempting to get data, counter = 1
  Downloaded data
  Time elapsed: 21.3 seconds
drive/MyDrive/Colab Notebooks/glorys/glorys_2020_06.nc
  Attempting to get data, counter = 1
  Downloaded data
  Time elapsed: 40.5 seconds
drive/MyDrive/Colab Notebooks/glorys/glorys_2020_07.nc
  Attempting to get data, counter = 1
  Downloaded data
  Time elapse